Interpretación de fechas y razonamiento cronológico

In [11]:
import os

import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()

db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [28]:
def gpt_system_user(
    system_message: str, user_message: str, model: str = "gpt-3.5-turbo"
):
    """Para usar en notebooks"""
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://api.openai.com/v1/chat/completions",
            params={
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
            data=json.dumps(
                {
                    "model": model,
                    "messages": [
                        {"content": system_message, "role": "system"},
                        {"content": user_message, "role": "user"},
                    ]
                }
            ),
            timeout=10,
        )
        choices = response.json()["choices"]
        answer = {
            "status_code": response.status_code,
            "text": choices[0]["message"],
        }
        return answer

    except requests.exceptions.RequestException:
        print("HTTP Request failed")
        return None


In [14]:
condensed_contexts =  [
        {
          "messages": [
              {
                "role": "user",
                "content": "Quiero un turno para mi madre"
              },
              {
                "role": "system",
                "content": "cual es el nombre de tu madre?"
              },
              {
                "role": "user",
                "content": "Alba"
              },
              {
                "role": "system",
                "content": "y que edad tiene?"
              },
              {
                "role": "user",
                "content": "66"
              },
              {
                  "role": "system",
                  "content": "perfecto, tiene turno para el 30 de agosto a las 15:00"
              }
           ],
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "context_id": "1",
            "created_at": "20230825T12:00:00.000Z",
          },
        },
        {
            "messages": [
                {
                "role": "user",
                "content": "Cual es la capital de Colombia?"
                },
                {
                    "role": "system",
                    "content": "Bogota"
                }
            ],
            "metadata": {
                "session_id": "1",
                "person_id": "20",
                "context_id": "2",
                "created_at": "20230825T18:00:00.000Z",                
              }
        }
  ]


In [15]:
for context in condensed_contexts:
        # Join all messages in one string by a comma
        messages = ''.join([str(message) for message in context['messages']])
        print(messages)
        print(context['metadata'])
        db.add_texts(
            texts=[messages], 
            metadatas=[context['metadata']],
        )

{'role': 'user', 'content': 'Quiero un turno para mi madre'}{'role': 'system', 'content': 'cual es el nombre de tu madre?'}{'role': 'user', 'content': 'Alba'}{'role': 'system', 'content': 'y que edad tiene?'}{'role': 'user', 'content': '66'}{'role': 'system', 'content': 'perfecto, tiene turno para el 30 de agosto a las 15:00'}
{'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20230825T12:00:00.000Z'}
{'role': 'user', 'content': 'Cual es la capital de Colombia?'}{'role': 'system', 'content': 'Bogota'}
{'session_id': '1', 'person_id': '20', 'context_id': '2', 'created_at': '20230825T18:00:00.000Z'}


Preguntamos por la edad simplemente, para empezar

In [25]:
user_input_3 = "que edad tiene alba?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.

```{context_3}```

"""
answer_3 = gpt35_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: {'role': 'user', 'content': 'Quiero un turno para mi madre'}{'role': 'system', 'content': 'cual es el nombre de tu madre?'}{'role': 'user', 'content': 'Alba'}{'role': 'system', 'content': 'y que edad tiene?'}{'role': 'user', 'content': '66'}{'role': 'system', 'content': 'perfecto, tiene turno para el 30 de agosto a las 15:00'}, metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20230825T12:00:00.000Z'} }


'Alba tiene 66 años.'

Ahora vemos si GPT 3.5 puede razonar con la fecha

In [26]:
user_input_3 = "que edad tenía alba en 2020?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.

```{context_3}```

"""
answer_3 = gpt35_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: {'role': 'user', 'content': 'Quiero un turno para mi madre'}{'role': 'system', 'content': 'cual es el nombre de tu madre?'}{'role': 'user', 'content': 'Alba'}{'role': 'system', 'content': 'y que edad tiene?'}{'role': 'user', 'content': '66'}{'role': 'system', 'content': 'perfecto, tiene turno para el 30 de agosto a las 15:00'}, metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20230825T12:00:00.000Z'} }


'En el contexto de la conversación anterior, Alba tenía 66 años en 2020.'

Vemos que la respuesta de GPT 3.5 no entiende el concepto de edad y fecha : 'En el contexto de la conversación anterior, Alba tenía 66 años en 2020.'

Probamos lo mismo con GPT 4

In [31]:
user_input_3 = "que edad tenía alba en 2020?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.

```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3, model="gpt-4-0613")
answer_3["text"]["content"]


CONTEXT:{ message: {'role': 'user', 'content': 'Quiero un turno para mi madre'}{'role': 'system', 'content': 'cual es el nombre de tu madre?'}{'role': 'user', 'content': 'Alba'}{'role': 'system', 'content': 'y que edad tiene?'}{'role': 'user', 'content': '66'}{'role': 'system', 'content': 'perfecto, tiene turno para el 30 de agosto a las 15:00'}, metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20230825T12:00:00.000Z'} }


'Alba nació en 1966. Por lo tanto, en 2020, tenía 54 años.'

Peor todavia, tomo la edad como año de nacimiento. Despues estuvo bien en hacer las cuentas. Cosa que 3.5 no hizo.

In [ ]:
Es necesario crear una extracción de data en la base.